# Set up the environment

In [1]:
from azureml.core import Workspace
ws = Workspace.get(name='demo-aml',
                   subscription_id='YOUR-SUSCRIPTION-ID',
                   resource_group='demo-aml')

In [2]:
from azureml.core.environment import Environment
from azureml.core.conda_dependencies import CondaDependencies

# to install required packages
env = Environment('keras-env')
cd = CondaDependencies.create(pip_packages=['azureml-defaults', 'tensorflow', 'keras', 'numpy'],
                              conda_packages = ['scikit-learn', 'joblib'])

env.python.conda_dependencies = cd

# Register environment to re-use later
env.register(workspace = ws)

{
    "databricks": {
        "eggLibraries": [],
        "jarLibraries": [],
        "mavenLibraries": [],
        "pypiLibraries": [],
        "rcranLibraries": []
    },
    "docker": {
        "arguments": [],
        "baseDockerfile": null,
        "baseImage": "mcr.microsoft.com/azureml/openmpi3.1.2-ubuntu18.04:20210531.v1",
        "baseImageRegistry": {
            "address": null,
            "password": null,
            "registryIdentity": null,
            "username": null
        },
        "enabled": false,
        "platform": {
            "architecture": "amd64",
            "os": "Linux"
        },
        "sharedVolumes": true,
        "shmSize": null
    },
    "environmentVariables": {
        "EXAMPLE_ENV_VAR": "EXAMPLE_VALUE"
    },
    "inferencingStackVersion": null,
    "name": "keras-env",
    "python": {
        "baseCondaEnvironment": null,
        "condaDependencies": {
            "channels": [
                "anaconda",
                "conda-forge"
    

In [3]:
%%time

# Optional
image = env.build(ws)
image.wait_for_completion(show_output=True)

Image Build Status: Running

2021/07/22 20:22:43 Downloading source code...
2021/07/22 20:22:45 Finished downloading source code
2021/07/22 20:22:45 Creating Docker network: acb_default_network, driver: 'bridge'
2021/07/22 20:22:46 Successfully set up Docker network: acb_default_network
2021/07/22 20:22:46 Setting up Docker configuration...
2021/07/22 20:22:46 Successfully set up Docker configuration
2021/07/22 20:22:46 Logging in to registry: demoaml.azurecr.io
2021/07/22 20:22:47 Successfully logged into demoaml.azurecr.io
2021/07/22 20:22:47 Executing step ID: acb_step_0. Timeout(sec): 5400, Working directory: '', Network: 'acb_default_network'
2021/07/22 20:22:47 Scanning for dependencies...
2021/07/22 20:22:48 Successfully scanned dependencies
2021/07/22 20:22:48 Launching container with name: acb_step_0
Sending build context to Docker daemon  66.56kB

Step 1/18 : FROM mcr.microsoft.com/azureml/openmpi3.1.2-ubuntu18.04:20210531.v1@sha256:fd561d73953587b677447897aea1f59a20a3abc0102

# Create scoring script

In [10]:
import os
import json
import joblib
import numpy as np
from tensorflow.keras.models import load_model

def init():
    global model
    model_path = './outputs/model.h5'
    model = load_model(model_path)

def run(raw_data):
    
    data = np.array(json.loads(raw_data)['data'])
    # make prediction
    y_hat = model.predict(data)
    #Load Scaler
    sc_y = joblib.load('./outputs/std_scaler_y.bin')
    y_hat = sc_y.inverse_transform(y_hat)
    # Pounds to kg
    y_hat = np.multiply(y_hat, 0.454)

    # you can return any data type as long as it is JSON-serializable
    return y_hat.tolist()

In [7]:
init()

In [8]:
import pandas as pd
X_val=pd.read_json('./inputs/X_validation_data.json', orient="split")
X_val.head()
X_val = X_val.values

In [11]:
# Online prediction: send a random row from the validatition set to score
random_index = np.random.randint(0, len(X_val)-1)
input_data = "{\"data\": [" + str(list(X_val[random_index])) + "]}"
run(input_data)

[[3.2364978790283203]]

In [12]:
# Batch prediction
batch = json.dumps({"data": X_val.tolist()})
batch = bytes(batch, encoding='utf8')
y_hat = run(batch)
y_hat

[[3.298445701599121],
 [2.9817092418670654],
 [3.3578171730041504],
 [3.331136703491211],
 [3.348602533340454],
 [3.7402615547180176],
 [3.5690150260925293],
 [3.5528311729431152],
 [3.407287836074829],
 [3.5607173442840576],
 [3.2574117183685303],
 [2.686293840408325],
 [2.0003843307495117],
 [3.333914279937744],
 [3.54276967048645],
 [3.505178213119507],
 [2.3151023387908936],
 [2.9131882190704346],
 [3.556436538696289],
 [3.3609042167663574],
 [3.3711719512939453],
 [1.8899664878845215],
 [3.189362049102783],
 [3.571166515350342],
 [3.427823781967163],
 [3.522000551223755],
 [1.9331085681915283],
 [2.657297372817993],
 [3.2883729934692383],
 [2.694244146347046],
 [3.423649311065674],
 [3.4520294666290283],
 [3.350761890411377],
 [3.4014501571655273],
 [3.060363292694092],
 [3.1641178131103516],
 [3.4051811695098877],
 [3.464489698410034],
 [3.2424850463867188],
 [2.9991402626037598],
 [3.436347723007202],
 [3.4306821823120117],
 [3.7755744457244873],
 [3.5445618629455566],
 [2.86629

In [14]:
%%writefile score.py
import os
import json
import joblib
import numpy as np
from tensorflow.keras.models import load_model

def init():
    global model
    model_path = os.path.join(os.getenv('AZUREML_MODEL_DIR'), 'outputs/model.h5')
    model = load_model(model_path)

def run(raw_data):
    data = np.array(json.loads(raw_data)['data'])
    # make prediction
    y_hat = model.predict(data)

    sc_y = joblib.load(os.path.join(os.getenv('AZUREML_MODEL_DIR'), 'outputs/std_scaler_y.bin'))
    y_hat = sc_y.inverse_transform(y_hat)
    y_hat = np.multiply(y_hat, 0.454)

    # you can return any data type as long as it is JSON-serializable
    return y_hat.tolist()

Overwriting score.py
